# Data Processing
---
### First inspection of dataset, data filtering and cleaning

Creation: 05.02.2021

## Loading in the Dataset
---

In [ ]:
# load in the dataset into a numpy array
raw_data_accidents = 0
raw_data_vehicles = 0
raw_data_casualties = 0